# Module 10 Task 1 - Interaction with API's

The goal of this project is to interact with the football-data API, and to find out which teams are the best at holding on wins at home (winning at half-time and winning at the end) and which are the best at turning the score around in the second half, regardless of whether it's at home or away. For both cases, we're interested in knowing not only how many times they've done it, but above all the percentage of score they've turned around / kept.

Import libraries

In [1]:
import requests
import pandas as pd
import json
import numpy as np

Declare our variables and make the GET request to the API endpoint

In [2]:
# id of the competition we want to get our data from, in this case #2016 is the Championship England's 2nd Division
id = 2016
url = f"http://api.football-data.org/v4/competitions/{id}/matches"

# headers for the GET request, the need to pass our Authentication Token
headers = {"X-Auth-Token": "be7154bc799640df977f1223ab78ae8b"}

response = requests.get(url, headers=headers)
response = json.loads(response.text)

We get our matches data by converting our JSON or dictionary object into a pandas dataframe.

In [3]:
matches_df = pd.json_normalize(response["matches"])
#print(response)
matches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      557 non-null    int64  
 1   utcDate                 557 non-null    object 
 2   status                  557 non-null    object 
 3   matchday                556 non-null    float64
 4   stage                   557 non-null    object 
 5   group                   0 non-null      object 
 6   lastUpdated             557 non-null    object 
 7   referees                557 non-null    object 
 8   area.id                 557 non-null    int64  
 9   area.name               557 non-null    object 
 10  area.code               557 non-null    object 
 11  area.flag               557 non-null    object 
 12  competition.id          557 non-null    int64  
 13  competition.name        557 non-null    object 
 14  competition.code        557 non-null    ob

There are columns that we don't need for our calculations, so we delete them.

In [4]:
# We type the name of the columns we are droping, we also specify that the removal is to axis 1 which are the columns

matches_df = matches_df.drop(["stage", "utcDate","group", "lastUpdated", "referees", "area.id", "area.name", "status", 'area.code', 'area.flag',
       'competition.id','competition.code', "competition.name", "matchday",
       'competition.type', 'competition.emblem', 'season.id',
       'season.startDate', 'season.endDate', 'season.currentMatchday','homeTeam.crest','awayTeam.crest', 'score.duration',
       'season.winner','homeTeam.name','awayTeam.name','odds.msg', "awayTeam.tla", "homeTeam.tla"], axis=1)

matches_df

,id,homeTeam.id,homeTeam.shortName,awayTeam.id,awayTeam.shortName,score.winner,score.fullTime.home,score.fullTime.away,score.halfTime.home,score.halfTime.away
0,436675,345.0,Sheffield Wed,340.0,Southampton,AWAY_TEAM,1.0,2.0,0.0,1.0
1,436676,59.0,Blackburn,74.0,West Brom,HOME_TEAM,2.0,1.0,2.0,0.0
2,436677,387.0,Bristol City,1081.0,Preston NE,DRAW,1.0,1.0,0.0,0.0
3,436678,343.0,Middlesbrough,384.0,Millwall,AWAY_TEAM,0.0,1.0,0.0,0.0
4,436679,68.0,Norwich,322.0,Hull City,HOME_TEAM,2.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
552,495747,68.0,Norwich,341.0,Leeds United,DRAW,0.0,0.0,0.0,0.0
553,495748,74.0,West Brom,340.0,Southampton,DRAW,0.0,0.0,0.0,0.0
554,495749,341.0,Leeds United,68.0,Norwich,None,NaN,NaN,NaN,NaN
555,495750,340.0,Southampton,74.0,West Brom,None,NaN,NaN,NaN,NaN


We want to calculate for each team, the times that they hold a winning score during half time and final time, and also calculate the times each time was able to turn the score around on the second half.

I will create another dataframe that will hold these values, where each row will be a team, and each column would be HomeWins, and 2ndHalfWins. We also need the percentage, so we will have columns for that too.

First, we get the team names

In [5]:
unique_teams = matches_df["homeTeam.shortName"].unique()
# Delete the last item in the list since its a "none" value
unique_teams = np.delete(unique_teams,-1)

print(unique_teams)



['Sheffield Wed' 'Blackburn' 'Bristol City' 'Middlesbrough' 'Norwich'
 'Plymouth Arg' 'Watford' 'Stoke' 'Swansea' 'Leicester City'
 'Leeds United' 'Sunderland' 'Coventry City' 'Birmingham' 'Cardiff'
 'Millwall' 'Southampton' 'Huddersfield' 'Hull City' 'Ipswich Town'
 'Preston NE' 'Rotherham Utd' 'West Brom' 'QPR']


Next, we create the DF

In [6]:
teams_df = pd.DataFrame(index=unique_teams, columns=["HomeWins", "%HomeWins","2ndHalfTurnArounds","%2ndHalfTurnArounds" ])

teams_df = teams_df.fillna(0)

teams_df.head()

/var/folders/6v/mydq28cs5jsf5r12rn8vl4vr0000gn/T/ipykernel_2414/157524392.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  teams_df = teams_df.fillna(0)


,HomeWins,%HomeWins,2ndHalfTurnArounds,%2ndHalfTurnArounds
Sheffield Wed,0,0,0,0
Blackburn,0,0,0,0
Bristol City,0,0,0,0
Middlesbrough,0,0,0,0
Norwich,0,0,0,0


We need to populate this table by doing a for loop, iterate through all the matches and calculate the values.

In [7]:
for index, game in matches_df.iterrows():
    home_team = game["homeTeam.shortName"]
    away_team = game["awayTeam.shortName"]
    HT_home_score = game["score.halfTime.home"]
    HT_away_score = game["score.halfTime.away"]
    FT_home_score = game["score.fullTime.home"]
    FT_away_score = game["score.fullTime.away"]

    #If the game ends in a draw, it won't be useful for our calculations. So we continue on the for loop.
    if(game["score.winner"] == "DRAW"):
      continue

    # Calculate the teams that hold wins on the first half and second half at home
    if( (HT_home_score > HT_away_score) and ( game["score.winner"] == "HOME_TEAM" ) ):
      #Update the HomeWins column on the teams_df
      teams_df.loc[home_team,'HomeWins'] = teams_df.loc[home_team, 'HomeWins'] + 1

    # Calculate the teams that turn the score around in the second half. We care about the home team and away team.
    if( (HT_home_score < HT_away_score) and (game["score.winner"] == "HOME_TEAM")):
      teams_df.loc[home_team,"2ndHalfTurnArounds"] = teams_df.loc[home_team, "2ndHalfTurnArounds"] + 1
    elif( (HT_away_score < HT_home_score) and (game["score.winner"] == "AWAY_TEAM") ):
      teams_df.loc[away_team,"2ndHalfTurnArounds"] = teams_df.loc[away_team, "2ndHalfTurnArounds"] + 1

Show Home Wins values.

In [8]:
print(teams_df.loc[:,['HomeWins']].sort_values('HomeWins',ascending=False))

                HomeWins
West Brom             11
Leeds United          11
Southampton           11
Ipswich Town          10
Norwich                8
Leicester City         8
Millwall               6
Bristol City           6
Middlesbrough          6
Plymouth Arg           6
Sheffield Wed          5
Preston NE             5
Coventry City          5
Sunderland             5
Stoke                  5
Birmingham             4
Blackburn              4
Huddersfield           4
Hull City              4
Swansea                4
QPR                    4
Cardiff                3
Rotherham Utd          2
Watford                2


Show 2nd half turn arounds values.

In [9]:
print(teams_df.loc[:,["2ndHalfTurnArounds"]].sort_values("2ndHalfTurnArounds",ascending=False))

                2ndHalfTurnArounds
Norwich                          3
Coventry City                    2
Bristol City                     2
Preston NE                       2
Ipswich Town                     2
Hull City                        2
Southampton                      2
Cardiff                          2
Leeds United                     2
Birmingham                       1
West Brom                        1
Sheffield Wed                    1
Sunderland                       1
Leicester City                   1
Swansea                          1
Stoke                            1
Watford                          1
QPR                              1
Blackburn                        0
Millwall                         0
Huddersfield                     0
Plymouth Arg                     0
Rotherham Utd                    0
Middlesbrough                    0


Next, we need to calculate the percentage of HomeWins and 2ndHalfTurnArounds. We would first need to find the total games for each team, we are going to add a new column in our teams dataframe.

This was automated since there could be teams that play more games than others, since there are leagues that contain playoffs.

In [10]:
#by using the stack and value_counts methods, we can find out the total games played by each team
total_matches_df = matches_df[["homeTeam.shortName", "awayTeam.shortName"]].stack().value_counts()
total_matches_df

West Brom         48
Leeds United      48
Norwich           48
Southampton       48
Sheffield Wed     46
Stoke             46
Sunderland        46
Cardiff           46
Coventry City     46
Leicester City    46
Birmingham        46
Swansea           46
Rotherham Utd     46
Watford           46
QPR               46
Huddersfield      46
Plymouth Arg      46
Hull City         46
Millwall          46
Middlesbrough     46
Preston NE        46
Bristol City      46
Blackburn         46
Ipswich Town      46
Name: count, dtype: int64

Now we merge columns

In [11]:
teams_df = teams_df.join(total_matches_df)

# Rename columnt count to matches
teams_df.rename(columns={'count':'matches'}, inplace=True)
teams_df

,HomeWins,%HomeWins,2ndHalfTurnArounds,%2ndHalfTurnArounds,matches
Sheffield Wed,5,0,1,0,46
Blackburn,4,0,0,0,46
Bristol City,6,0,2,0,46
Middlesbrough,6,0,0,0,46
Norwich,8,0,3,0,48
Plymouth Arg,6,0,0,0,46
Watford,2,0,1,0,46
Stoke,5,0,1,0,46
Swansea,4,0,1,0,46
Leicester City,8,0,1,0,46


Now we calculate the percentage for both HomeWins and 2ndHalfTurnArounds

In [12]:
teams_df["%HomeWins"] = round((teams_df["HomeWins"]/teams_df["matches"]) * 100 , 2)
teams_df["%2ndHalfTurnArounds"] = round((teams_df["2ndHalfTurnArounds"]/teams_df["matches"]) * 100 ,2)

teams_df

,HomeWins,%HomeWins,2ndHalfTurnArounds,%2ndHalfTurnArounds,matches
Sheffield Wed,5,10.87,1,2.17,46
Blackburn,4,8.70,0,0.00,46
Bristol City,6,13.04,2,4.35,46
Middlesbrough,6,13.04,0,0.00,46
Norwich,8,16.67,3,6.25,48
Plymouth Arg,6,13.04,0,0.00,46
Watford,2,4.35,1,2.17,46
Stoke,5,10.87,1,2.17,46
Swansea,4,8.70,1,2.17,46
Leicester City,8,17.39,1,2.17,46


Display the top 5 teams with best percentage of holding a positive score at half time and final time.

In [13]:
teams_df["%HomeWins"].sort_values(ascending=False).head()

West Brom         22.92
Leeds United      22.92
Southampton       22.92
Ipswich Town      21.74
Leicester City    17.39
Name: %HomeWins, dtype: float64

Display the top 5 teams with the best percentage of turning the score around at the second half

In [14]:
teams_df["%2ndHalfTurnArounds"].sort_values(ascending=False).head()

Norwich          6.25
Coventry City    4.35
Bristol City     4.35
Preston NE       4.35
Ipswich Town     4.35
Name: %2ndHalfTurnArounds, dtype: float64